In [2]:
import os
import requests
import pandas as pd
from tqdm import tqdm
import ast
import time

# -------------------------------------------------------------
# Paths
# -------------------------------------------------------------
coord_path = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\structure_coordinates.csv"
out_dir = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data"
os.makedirs(out_dir, exist_ok=True)

# -------------------------------------------------------------
# Load coordinates
# -------------------------------------------------------------
df = pd.read_csv(coord_path)

# Sample size option (set to None to process all coordinates)
sample_size = 10  # Change to None for full dataset

def parse_coord(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else x
    except:
        return None

df["COORDINATES"] = df["COORDINATES"].apply(parse_coord)
df = df[df["COORDINATES"].notna()].copy()

# Apply sample size if specified
if sample_size is not None:
    df = df.head(sample_size)
    print(f"Processing sample of {len(df)} coordinates (sample_size={sample_size})")
else:
    print(f"Processing all {len(df)} coordinates")

# -------------------------------------------------------------
# Quiet GET wrapper
# -------------------------------------------------------------
def safe_get(url, params=None):
    try:
        r = requests.get(url, params=params, timeout=12)
        if r.status_code == 200:
            return r.json()
        return None
    except:
        return None

# -------------------------------------------------------------
# FEMA NFHL Fetcher
# -------------------------------------------------------------
def fetch_nfhl(lat, lon):

    # Small bounding box around the structure
    delta = 0.0001  # ≈ 11 meters
    xmin = lon - delta
    xmax = lon + delta
    ymin = lat - delta
    ymax = lat + delta

    url = "https://hazards.fema.gov/arcgis/rest/services/public/NFHL/MapServer"

    params = {
        "service": "WFS",
        "version": "2.0.0",
        "request": "GetFeature",
        "typeNames": "floodHazardZones",
        "outputFormat": "json",
        "bbox": f"{xmin},{ymin},{xmax},{ymax},EPSG:4326"
    }

    js = safe_get(url, params)
    if not js or "features" not in js or not js["features"]:
        # No flood zone at this location → default nulls
        return None

    feature = js["features"][0]
    props = feature.get("properties", {})

    return {
        "NFHL_FLD_ZONE": props.get("FLD_ZONE"),
        "NFHL_SFHA": props.get("SFHA_TF"),
        "NFHL_STATIC_BFE": props.get("STATIC_BFE"),
        "NFHL_V_ZONE": props.get("V_ZONE"),
        "NFHL_ZONE_SUBTYPE": props.get("ZONE_SUBTYPE"),
        "NFHL_SOURCE_CIT": props.get("SOURCE_CIT")
    }

# -------------------------------------------------------------
# Loop all structures
# -------------------------------------------------------------
rows = []
print("\nStarting FEMA NFHL enrichment...\n")

for _, r in tqdm(df.iterrows(), total=len(df), desc="FEMA NFHL"):
    sid = r["STRUCTURE_ID"]
    lat, lon = r["COORDINATES"]

    entry = {
        "STRUCTURE_ID": sid,
        "NFHL_FLD_ZONE": None,
        "NFHL_SFHA": None,
        "NFHL_STATIC_BFE": None,
        "NFHL_V_ZONE": None,
        "NFHL_ZONE_SUBTYPE": None,
        "NFHL_SOURCE_CIT": None
    }

    result = fetch_nfhl(lat, lon)
    if result:
        entry.update(result)

    rows.append(entry)
    time.sleep(0.15)   # FEMA WFS can be touchy → slight delay recommended

# -------------------------------------------------------------
# Save CSV
# -------------------------------------------------------------
df_out = pd.DataFrame(rows)
out_path = os.path.join(out_dir, "fema_nfhl_enrichment.csv")
df_out.to_csv(out_path, index=False)

print(f"\nSaved FEMA NFHL flood hazard enrichment for {len(df_out)} structures:")
print(out_path)

Processing sample of 10 coordinates (sample_size=10)

Starting FEMA NFHL enrichment...



FEMA NFHL:   0%|          | 0/10 [00:00<?, ?it/s]

FEMA NFHL: 100%|██████████| 10/10 [00:07<00:00,  1.41it/s]


Saved FEMA NFHL flood hazard enrichment for 10 structures:
C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\fema_nfhl_enrichment.csv
